In [1]:
import ROOT
ROOT.EnableImplicitMT()

In [2]:
dataset_path = "root://eospublic.cern.ch//eos/opendata/atlas/OutreachDatasets/2020-01-22"

import json
files = json.load( open( "WBosonAnalysis.json" ) )

lumi = 10064.0
print( 'Run on data corresponding to {:.2f} fb^-1'.format( lumi / 1000.0 ) )

Run on data corresponding to 10.06 fb^-1


In [3]:
processes = files.keys()
df = {}
xsecs = {}
sumws = {}
samples = []
for p in processes:
    for d in files[p]:
        # Construct the dataframes
        folder = d[0] # Folder name
        sample = d[1] # Sample name
        xsecs[sample] = d[2] # Cross-section
        sumws[sample] = d[3] # Sum of weights
        num_events = d[4] # Number of events
        samples.append(sample)
        df[sample] = ROOT.RDataFrame("mini", "{}/1lep/{}/{}.1lep.root".format(dataset_path, folder, sample))
        # # Scale down the datasets if requested
        # if args.full_dataset and lumi_scale < 1.0:
        #     df[sample] = df[sample].Range(int(num_events * lumi_scale))
df        

{'data_A': <cppyy.gbl.ROOT.RDataFrame object at 0xbd76c50>,
 'data_B': <cppyy.gbl.ROOT.RDataFrame object at 0x7fd0540024a0>,
 'data_C': <cppyy.gbl.ROOT.RDataFrame object at 0x7fd060003fa0>,
 'data_D': <cppyy.gbl.ROOT.RDataFrame object at 0x7fd050003310>,
 'mc_364156.Wmunu_PTV0_70_CVetoBVeto': <cppyy.gbl.ROOT.RDataFrame object at 0x7fd0540065d0>,
 'mc_364157.Wmunu_PTV0_70_CFilterBVeto': <cppyy.gbl.ROOT.RDataFrame object at 0x7fd060004cd0>,
 'mc_364158.Wmunu_PTV0_70_BFilter': <cppyy.gbl.ROOT.RDataFrame object at 0x7fd060006320>,
 'mc_364159.Wmunu_PTV70_140_CVetoBVeto': <cppyy.gbl.ROOT.RDataFrame object at 0x7fd060003720>,
 'mc_364160.Wmunu_PTV70_140_CFilterBVeto': <cppyy.gbl.ROOT.RDataFrame object at 0x7fd060002af0>,
 'mc_364161.Wmunu_PTV70_140_BFilter': <cppyy.gbl.ROOT.RDataFrame object at 0x7fd0600068b0>,
 'mc_364162.Wmunu_PTV140_280_CVetoBVeto': <cppyy.gbl.ROOT.RDataFrame object at 0x7fd0600053b0>,
 'mc_364163.Wmunu_PTV140_280_CFilterBVeto': <cppyy.gbl.ROOT.RDataFrame object at 0x7fd0

In [4]:
# Select events for the analysis

# Just-in-time compile custom helper function performing complex computations
ROOT.gInterpreter.Declare("""
bool GoodElectronOrMuon(int type, float pt, float eta, float phi, float e, float trackd0pv, float tracksigd0pv, float z0)
{
    ROOT::Math::PtEtaPhiEVector p(pt / 1000.0, eta, phi, e / 1000.0);
    if (abs(z0 * sin(p.theta())) > 0.5) return false;
    if (type == 11 && abs(eta) < 2.46 && (abs(eta) < 1.37 || abs(eta) > 1.52)) {
        if (abs(trackd0pv / tracksigd0pv) > 5) return false;
        return true;
    }
    if (type == 13 && abs(eta) < 2.5) {
        if (abs(trackd0pv / tracksigd0pv) > 3) return false;
        return true;
    }
    return false;
}
""")

for s in samples:
    # Select events with a muon or electron trigger and with a missing transverse energy larger than 30 GeV
    df[s] = df[s].Filter("trigE || trigM")\
                 .Filter("met_et > 30000")

    # Find events with exactly one good lepton
    df[s] = df[s].Define("good_lep", "lep_isTightID && lep_pt > 35000 && lep_ptcone30 / lep_pt < 0.1 && lep_etcone20 / lep_pt < 0.1")\
                 .Filter("ROOT::VecOps::Sum(good_lep) == 1")

    # Apply additional cuts in case the lepton is an electron or muon
    df[s] = df[s].Define("idx", "ROOT::VecOps::ArgMax(good_lep)")\
                 .Filter("GoodElectronOrMuon(lep_type[idx], lep_pt[idx], lep_eta[idx], lep_phi[idx], lep_E[idx], lep_trackd0pvunbiased[idx], lep_tracksigd0pvunbiased[idx], lep_z0[idx])")

# Apply luminosity, scale factors and MC weights for simulated events
for s in samples:
    if "data" in s:
        df[s] = df[s].Define("weight", "1.0")
    else:
        df[s] = df[s].Define("weight", "scaleFactor_ELE * scaleFactor_MUON * scaleFactor_LepTRIGGER * scaleFactor_PILEUP * mcWeight * {} / {} * {}".format(xsecs[s], sumws[s], lumi))

# Compute transverse mass of the W boson using the lepton and the missing transverse energy and make a histogram
ROOT.gInterpreter.Declare("""
float ComputeTransverseMass(float met_et, float met_phi, float lep_pt, float lep_eta, float lep_phi, float lep_e)
{
    ROOT::Math::PtEtaPhiEVector met(met_et, 0, met_phi, met_et);
    ROOT::Math::PtEtaPhiEVector lep(lep_pt, lep_eta, lep_phi, lep_e);
    return (met + lep).Mt() / 1000.0;
}
""")

histos = {}
for s in samples:
    df[s] = df[s].Define("mt_w", "ComputeTransverseMass(met_et, met_phi, lep_pt[idx], lep_eta[idx], lep_phi[idx], lep_E[idx])")
    histos[s] = df[s].Histo1D(ROOT.RDF.TH1DModel(s, "mt_w", 24, 60, 180), "mt_w", "weight")
histos

{'data_A': <cppyy.gbl.ROOT.RDF.RResultPtr<TH1D> object at 0x3619f2040>,
 'data_B': <cppyy.gbl.ROOT.RDF.RResultPtr<TH1D> object at 0x2b175c140>,
 'data_C': <cppyy.gbl.ROOT.RDF.RResultPtr<TH1D> object at 0x327e558b0>,
 'data_D': <cppyy.gbl.ROOT.RDF.RResultPtr<TH1D> object at 0x318edf7b0>,
 'mc_364156.Wmunu_PTV0_70_CVetoBVeto': <cppyy.gbl.ROOT.RDF.RResultPtr<TH1D> object at 0x177afe790>,
 'mc_364157.Wmunu_PTV0_70_CFilterBVeto': <cppyy.gbl.ROOT.RDF.RResultPtr<TH1D> object at 0x318f16d60>,
 'mc_364158.Wmunu_PTV0_70_BFilter': <cppyy.gbl.ROOT.RDF.RResultPtr<TH1D> object at 0x361a36070>,
 'mc_364159.Wmunu_PTV70_140_CVetoBVeto': <cppyy.gbl.ROOT.RDF.RResultPtr<TH1D> object at 0x2f5763ef0>,
 'mc_364160.Wmunu_PTV70_140_CFilterBVeto': <cppyy.gbl.ROOT.RDF.RResultPtr<TH1D> object at 0x265238be0>,
 'mc_364161.Wmunu_PTV70_140_BFilter': <cppyy.gbl.ROOT.RDF.RResultPtr<TH1D> object at 0x2f588dd90>,
 'mc_364162.Wmunu_PTV140_280_CVetoBVeto': <cppyy.gbl.ROOT.RDF.RResultPtr<TH1D> object at 0x2a3a684d0>,
 'mc_

In [ ]:
# Run the event loop and merge histograms of the respective processes

# RunGraphs allows to run the event loops of the separate RDataFrame graphs
# concurrently. This results in an improved usage of the available resources
# if each separate RDataFrame can not utilize all available resources, e.g.,
# because not enough data is available.
ROOT.RDF.RunGraphs([histos[s] for s in samples])

In [ ]:
def merge_histos(label):
    h = None
    for i, d in enumerate(files[label]):
        t = histos[d[1]].GetValue()
        if i == 0: h = t.Clone()
        else: h.Add(t)
    h.SetNameTitle(label, label)
    return h

data = merge_histos("data")
wjets = merge_histos("wjets")
zjets = merge_histos("zjets")
ttbar = merge_histos("ttbar")
diboson = merge_histos("diboson")
singletop = merge_histos("singletop")

In [ ]:
# Set styles
ROOT.gROOT.SetStyle("ATLAS")

# Create canvas
c = ROOT.TCanvas("c", "", 600, 600)
c.SetTickx(0)
c.SetTicky(0)
c.SetLogy()

# Draw stack with MC contributions
stack = ROOT.THStack()
for h, color in zip(
        [singletop, diboson, ttbar, zjets, wjets],
        [(208, 240, 193), (195, 138, 145), (155, 152, 204), (248, 206, 104), (222, 90, 106)]):
    h.SetLineWidth(1)
    h.SetLineColor(1)
    h.SetFillColor(ROOT.TColor.GetColor(*color))
    stack.Add(h)
stack.Draw("HIST")
stack.GetXaxis().SetLabelSize(0.04)
stack.GetXaxis().SetTitleSize(0.045)
stack.GetXaxis().SetTitleOffset(1.3)
stack.GetXaxis().SetTitle("m_{T}^{W#rightarrow l#nu} [GeV]")
stack.GetYaxis().SetTitle("Events")
stack.GetYaxis().SetLabelSize(0.04)
stack.GetYaxis().SetTitleSize(0.045)
stack.SetMaximum(1e10)
stack.SetMinimum(1)

# Draw data
data.SetMarkerStyle(20)
data.SetMarkerSize(1.2)
data.SetLineWidth(2)
data.SetLineColor(ROOT.kBlack)
data.Draw("E SAME")

# Add legend
legend = ROOT.TLegend(0.60, 0.65, 0.92, 0.92)
legend.SetTextFont(42)
legend.SetFillStyle(0)
legend.SetBorderSize(0)
legend.SetTextSize(0.04)
legend.SetTextAlign(32)
legend.AddEntry(data, "Data" ,"lep")
legend.AddEntry(wjets, "W+jets", "f")
legend.AddEntry(zjets, "Z+jets", "f")
legend.AddEntry(ttbar, "t#bar{t}", "f")
legend.AddEntry(diboson, "Diboson", "f")
legend.AddEntry(singletop, "Single top", "f")
legend.Draw()

# Add ATLAS label
text = ROOT.TLatex()
text.SetNDC()
text.SetTextFont(72)
text.SetTextSize(0.045)
text.DrawLatex(0.21, 0.86, "ATLAS")
text.SetTextFont(42)
text.DrawLatex(0.21 + 0.16, 0.86, "Open Data")
text.SetTextSize(0.04)
# text.DrawLatex(0.21, 0.80, "#sqrt{{s}} = 13 TeV, {:.2f} fb^{{-1}}".format(lumi * args.lumi_scale / 1000.0))
text.DrawLatex( 0.21, 0.80, "#sqrt{{s}} = 13 TeV, {:.2f} fb^{{-1}}".format( lumi / 1000.0 ) )

# c.Draw()

In [ ]:
c.Draw()